In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

## Wyznaczenie ceny za pomocą Monte Carlo

## Szacowanie pochodnej w punkcie

In [2]:
class Underlying:
    def __init__(self, mu, sigma, r):
        self.mu = mu
        self.sigma = sigma
        self.r = r
    def simulate_P(self, size, T, time_stamps = 100):
        time = np.arange(0,time_stamps)/(time_stamps-1) * T
        B = np.random.normal(size=(size, time_stamps), loc=0, scale=1)
        W = time * B
        sims = np.exp((self.mu - 0.5*self.sigma**2)*time + self.sigma*W)
        return (B,sims)
    def simulate_Q(self, size, T, time_stamps = 100):
        time = np.arange(0,time_stamps)/(time_stamps-1) * T
        B = np.random.normal(size=(size, time_stamps), loc=0, scale=1)
        W = time * B
        sims = np.exp((self.r - 0.5*self.sigma**2)*time + self.sigma*W)
        return (B, sims)

In [12]:
class Option:
    def __init__(self, underlying, payoff_func, T):
        self.underlying = underlying
        self.payoff_func = payoff_func
        self.T = T
    def MC_price(self, X0, r, n_sims, method='crude'):
        discount = np.exp(-r*self.T)
        B, sims = self.underlying.simulate_Q(n_sims, self.T)
        payoffs = self.payoff_func(X0*sims)
        payoffs_mean = payoffs.mean()
        if method == 'crude':
            price = payoffs_mean
        elif method == 'var_control':
            MC_B = B[:,-1].mean()
            rho = np.sum((payoffs-payoffs_mean)*(B[:,-1]-MC_B))/(payoffs.shape[0]-1)
            price = payoffs_mean - rho/1 * (MC_B - 0)
        else:
            raise Exception(f'Method {method} not implemented...')
        return discount * price
    def MC_delta(self, dX, r, n_sims, method='crude'):
        price_minus = self.MC_price(X0-dX, r, n_sims, method)
        price_plus = self.MC_price(X0+dX, r, n_sims, method)
        delta = (price_plus - price_minus)/(2*dX)
        return delta

In [13]:
def payoff_call(X, K):
    return np.maximum(X[:,-1]- K, 0)
def payoff_put(X, K):
    return np.maximum(K - X[:,-1], 0)

In [21]:
x=np.array([[1,2,3],[2,4,5]])

In [25]:
def payoff_knock_out_call(X, K, barrier):
    safe = (np.max(X,axis=1)>=barrier) * 1
    payoff = np.maximum(X[:,-1] - K, 0) * safe
    return payoff

In [29]:
n_sims = 1000000
X0 = 100
dX = 10
mu = 0.1
r = 0.05
sigma = 0.1
T = 1
K = 100
up_barrier = 120

In [15]:
underlying = Underlying(mu, sigma, r)

In [16]:
vanilla_call = Option(underlying, lambda X: payoff_call(X, K), T)

In [17]:
vanilla_call.MC_price(X0, r, n_sims, method='crude')

6.792299815664246

In [18]:
vanilla_call.MC_price(X0, r, n_sims, method='var_control')

6.804115693958655

In [19]:
vanilla_call.MC_delta(dX, r, n_sims, method='crude')

0.6767405691194467

In [20]:
vanilla_call.MC_delta(dX, r, n_sims, method='var_control')

0.6762655863324376

In [30]:
knock_out_call = Option(underlying, lambda X: payoff_knock_out_call(X, K, up_barrier), T)

In [31]:
knock_out_call.MC_price(X0, r, n_sims, method='crude')

5.768639256216623

In [32]:
knock_out_call.MC_price(X0, r, n_sims, method='var_control')

5.753413285593999

In [33]:
knock_out_call.MC_delta(dX, r, n_sims, method='crude')

0.7466325587044154

In [34]:
knock_out_call.MC_delta(dX, r, n_sims, method='var_control')

0.746704087936571